<a href="https://colab.research.google.com/github/doctorsmylie/mtg-draft-agent/blob/main/17landsdataimport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Notebook takes the 17Lands Gameplay data, processes it, and seves it to an HDF (.h5) file in the same folder

In [ ]:
# Configure Drive or Jupyter notebook -- only runs when first loaded
if "CONFIG_DONE" not in globals():
    # Need to mount drive and clone repo to access data and functions
    try:
        from google.colab import drive  # type: ignore

        IN_COLAB = True

        # clone repo
        !git clone https://github.com/doctorsmylie/mtg-draft-agent
        %cd mtg-draft-agent

    except ModuleNotFoundError:
        IN_COLAB = False

    # Finish configuration -- also configures notebook outside of Colab
    %run "project_path.ipynb"
else:
    print("Config done already")

Cloning into 'mtg-draft-agent'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 93 (delta 38), reused 57 (delta 23), pack-reused 0 (from 0)
Receiving objects: 100% (93/93), 193.56 KiB | 4.61 MiB/s, done.
Resolving deltas: 100% (38/38), done.
/content/mtg-draft-agent
Starting config...
Running in Colab? Yes

Configuring Google Colab...
Mounting Drive...
Mounted at /content/mtg-draft-agent/drive
BASE_PATH =  /content/mtg-draft-agent
DATA_FOLDER = /content/mtg-draft-agent/drive/MyDrive/Erdos25/MTGdraft
BASE_PATH == os.getcwd(): True

Configuration done


In [ ]:
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import polars as pl

# For managing paths
import pathlib
import itertools

In [ ]:
%pip install spells-mtg
import spells
import shutil
import os
import tqdm

Loading data

In [ ]:
#expansion code
expansion = 'DSK'
#root directiory for spells library
root_directory =  pathlib.Path('/root/.local/share/spells/external',expansion)
#drive directory for spells data
drive_directory = pathlib.Path(DATA_FOLDER,'spells_data',expansion)

In [ ]:
#Copy Spells data to the root folder
source_directory = drive_directory
destination_directory = root_directory
try:
    shutil.copytree(source_directory, destination_directory)
    print(f"Directory '{source_directory}' copied successfully to '{destination_directory}'.")
except FileExistsError:
    print(f"Error: Destination directory '{destination_directory}' already exists.")
except Exception as e:
    print(f"An error occurred: {e}")

##Alternatively use command
#!spells add {expansion}

Directory '/content/mtg-draft-agent/drive/MyDrive/Erdos25/MTGdraft/spells_data/DSK' copied successfully to '/root/.local/share/spells/external/DSK'.


In [ ]:
#Use spells.download_data function to dowload/update data from 17lands
def spells_download_data(expansion, update=False, overwrite=False):
  #check if directory exists
  root_dir_data=pathlib.Path('/root/.local/share/spells/external',expansion)
  if not root_dir_data.exists():
    !spells add {expansion}
  elif update:
    !spells refresh {expansion}
  else:
    print(f"Directory '{root_dir_data}' already exists.")
    print(f"to update set update=True")
  # copy directory to DATA_FOLDER
  source_directory = root_dir_data
  destination_directory = pathlib.Path(DATA_FOLDER,'spells_data',expansion)
  try:
    shutil.copytree(source_directory, destination_directory)
    print(f"Directory '{source_directory}' copied successfully to '{destination_directory}'.")
  except FileExistsError:
    if overwrite:
      shutil.rmtree(destination_directory)
      shutil.copytree(source_directory, destination_directory)
      print(f"Directory '{source_directory}' copied successfully to '{destination_directory}'.")
    else:
      print(f"Error: Destination directory '{destination_directory}' already exists.")
      print(f"to overwrite set overwrite=True")
  except OSError as e:
    print(f"An error occurred: {e}")

In [ ]:
def load_gamedata(expansion):
  """
  Loads game data for a given expansion.

  Args:
    expansion: The expansion code (e.g., 'DSK').

    Returns:
    A polars lazyframe
  """
  gamefilename=expansion+'_PremierDraft_game.parquet'
  game_file = pathlib.Path(drive_directory, gamefilename)
  gamedata = pl.scan_parquet(game_file)
  return gamedata
def group_gamedata(gamedata):
  """
  Groups game data by draft_id and build_index.

  Args:
    gamedata: A polars lazyframe.

    Returns:
    A polars lazyframe
    """
  grouped_gamedata = gamedata.group_by(['draft_id', 'build_index'])
  return grouped_gamedata

gamedata = load_gamedata(expansion)
grouped_gamedata = group_gamedata(gamedata)

In [ ]:
# Column lists
column_list_with_basic_lands=gamedata.collect_schema().names()
basic_land_attributes= [col for col in column_list_with_basic_lands if col.endswith('Plains') or col.endswith('Island')
                                                    or col.endswith('Swamp') or col.endswith('Mountain') or col.endswith('Forest')]
basic_land_deck_attributes= [col for col in basic_land_attributes if col.startswith('deck_')]
basic_land_sideboard_attributes= [col for col in basic_land_attributes if col.startswith('sideboard_')]
basic_land_opening_hand_attributes= [col for col in basic_land_attributes if col.startswith('opening_hand_')]
basic_land_drawn_attributes= [col for col in basic_land_attributes if col.startswith('drawn_')]
basic_land_tutored_attributes= [col for col in basic_land_attributes if col.startswith('tutored_')]
column_list= [col for col in column_list_with_basic_lands if not col.endswith('Plains') and not col.endswith('Island')
                                                    and not col.endswith('Swamp') and not col.endswith('Mountain') and not col.endswith('Forest')]
deck_atributes= [col for col in column_list if col.startswith('deck_')]
card_list= [col.replace('deck_', '') for col in deck_atributes]
sideboard_atributes= [col for col in column_list if col.startswith('sideboard_')]
opening_hand_atributes= [col for col in column_list if col.startswith('opening_hand_')]
drawn_atributes= [col for col in column_list if col.startswith('drawn_')]
tutored_atributes= [col for col in column_list if col.startswith('tutored_')]
id_atributes= ['draft_id','build_index']
user_atributes= [col for col in column_list if col.startswith('user_')]
colors_atributes= [col for col in column_list if col.endswith('_colors')]
won_atributes= ['won']
game_atributes= ['game_time','match_number', 'game_number','rank', 'opp_rank', 'on_play', 'num_mulligans',
                   'opp_num_mulligans', 'opp_colors', 'num_turns']
other_atributes= ['expansion', 'event_type', 'draft_time']

# Get columns that are not in any of the defined attribute lists
remaining_columns = [col for col in column_list if col not in deck_atributes + sideboard_atributes + opening_hand_atributes + drawn_atributes + tutored_atributes + id_atributes + user_atributes + colors_atributes + won_atributes + game_atributes + other_atributes]

# Print the remaining columns to see what they are
print("Remaining columns:", remaining_columns)

Remaining columns: []


In [ ]:
#Aggregation rules list
aggregation_rules_win_rate = [
    pl.col('won').sum().alias('wins'),
    pl.col('won').count().alias('total_matches'),
    pl.col('won').mean().alias('win_rate').cast(pl.Float32)
]


aggregation_rules_basic_land_deck_atributes = [
    pl.col(col).first().alias(col) for col in basic_land_deck_attributes
]

aggregation_rules_user_atributes = [
    pl.col(col).first().alias(col) for col in user_atributes
]

aggregation_rules_colors_atributes = [
    pl.col(col).first().alias(col) for col in colors_atributes
]

aggregation_rules_deck_atributes = [
    pl.col(col).first().alias(col) for col in deck_atributes
]

aggregation_rules= (
                    aggregation_rules_win_rate
                    + aggregation_rules_basic_land_deck_atributes
                    + aggregation_rules_user_atributes
                    + aggregation_rules_colors_atributes
                    + aggregation_rules_deck_atributes
                   )

aggregation_rule_win_rate=(
                    (pl.col('wins').sum()/pl.col('total_matches').sum())
                    .alias('win_rate').cast(pl.Float32)
                    )

In [ ]:
deck_gamedata_wr = (grouped_gamedata
                    .agg(aggregation_rules)
                    .filter(pl.col('total_matches') > 2)
                    )
deck_gamedata_collected=deck_gamedata_wr.collect()